# Feature Entropy and Mutual information

The purpose of this script is to calculate the mutual information between the caseness variable (i.e. record of a SNOMED CT code for conditions indicative of complex mental health difficulties) and feature sets whose entropy is at least as great as the caseness variable's.

This Jupyter notebook runs "UNSEEN create feature set array.IPYNB" on which it depends, and "UNSEEN create caseness array.IPYNB" on which the feature-set Jupyter notebook depends.

### Imports

In [150]:
import itertools

## Dependencies

Run "UNSEEN create feature set array.IPYNB", which runs "UNSEEN create caseness array.ipynb".

In [151]:
%run ./"UNSEEN create feature set array.ipynb"

In [152]:
%run ./"UNSEEN create caseness array.ipynb"

Caseness variable entropy =  0.4 nats
Caseness variable scaled entropy =  57.7 %
Hit rate (all) = 13.7 %
Hit rate (none) = 86.3 %
Odds (No CMHD : CMHD) =  6.3 times less likely to have CMHD than to have it.


## Prerequisites

In [4]:
# Instantiate template for storing entropy.
feature_entropy_TEMPLATE =\
    pandas.DataFrame(columns = ['Feature set', 'Entropy'])

# Instantiate template for storing mutual information.
feature_mutual_information_TEMPLATE =\
    pandas.DataFrame(columns = ['Feature set', 'Mutual information'])

# Instantiate storage for features that are dropped due to low entropy.
f_to_drop_TEMPLATE =\
    pandas.DataFrame(columns = ['Dropped feature set'])

## Calculate the entropy and two-way mutual information of the feature sets and the caseness.

Our focus is mutual information but I check the entropy of features so that I don't calculate the mutual information for any features whose entropy is less than that of the caseness variable. Justification for this action is based on the fact that the mutual information between the caseness variable and any feature will be less than or equal to the lesser entropy of the caseness or feature, i.e. $I(X_{i};CMHD) ≤ min\{H(X_{i}), H(CMHD)\}$. We don’t want any feature set that is worse than no feature set (i.e. having only the caseness prevalence to predict a random outcome value) so we don’t bother with any feature set that will lower the possible mutual information. Two-way mutual information* will not be calculated for those features whose entropy is less than the caseness variable's entropy. The dropped variables are indicated in the `f_to_drop` pandas.DataFrame.

First, two-way mutual information will be calculated between the caseness variable and individual feature sets. Secondly, two-way mutual information will be calculated between the caseness variable and pair-composites of feature sets. These pair composites are individual feature sets that amalgamate two feature sets into a new binary definition, where values are `0` if both component feature sets are zero and `1` otherwise**. More-complicated encoding is possible, e.g. a different level for every combination of values from each component feature set. Further code extends these feature-set compositions up to quintuplet composites (i.e. amalgamating five feature sets into a single binary variable).

<br/>
<br/>

__\*__ _Initially, the plan was to use $k$-way mutual information for $k>2$ but the meaning of these mutual information values is controversial at best. I side with [Krippendorf's assessment](https://sci-hub.wf/10.1080/03081070902993160) , which renders 3-way mutual information interpretable but not any higher-order mutual information statistics (yet?). I decided to stick with two-way mutual information using composite feature sets so that I am comparing the same statistic across individual and composite feature sets._

__\**__ _Other encodings will be trialled at a later date. If a feature set contains more than one feature, then it will be represented in three ways: OR, AND, and multinomial. The OR representation (alternative called the at-least-one representation) is a binary variable with a value of `0` if the component features are all zero, and `1` otherwise. The AND (alternative called the all-present representation) representation is a binary variable with a value of `1` if all component features are one, and `0` otherwise. The multinomial representation is a multinomial variable with values for each of the possible combinations of component features’ values. For example, given a feature set of two features $A=\{0,1\}$ and $B=\{0,1\}$, their multinomial feature-set representation would be $C=\{0,1,2,3\}$, where $C=0=(A=0\   AND\   B=0)$, $C=1=(A=0\   AND\   B=1)$, $C=2=(A=1\  AND\  B=0)$, and $C=3=(A=1\   AND\   B=1)$._

### Define a function to calculate the mutual information between feature sets and the caseness variable.

In [243]:
# Define function that will calculate two-way mutual
# information for the features of order m.
def featuresetmi(featureArray,
                 m = None,
                 savelocation = None,
                 encoding = None):
    # ## Assess argument validty.
    #
    # Check order of feature set. If not provided,
    # default to m = 1.
    
    # match m:
    #     case None:
    #         order_int = 1
    #         order_label = "Individuals"
    #         print("\nNo value for m provided." +
    #               "\nDefault value of m = 1 will be used.")
    #     case 1:
    #         order_int = m
    #         order_label = "Individuals"
    #     case 2:
    #         order_int = m
    #         order_label = "Pairs"
    #     case 3:
    #         order_int = m
    #         order_label = "Triplets"
    #     case 4:
    #         order_int = m
    #         rder_label = "Quadruplets"
    #     case 5:
    #         order_int = m
    #         order_label = "Quintuplets"
    #     case _:
    #         print("\n** Error: Integer value between 1",
    #               "5 not supplied for m.**\n")
    #         return
    
    if m == None:
        order_int = 1
        order_label = "Individuals"
        print("\nNo value for m provided." +
              "\n...Default value of m = 1 will be used.")
    elif m == 1:
        order_int = m
        order_label = "Individuals"
    elif m == 2:
        order_int = m
        order_label = "Pairs"
    elif m == 3:
        order_int = m
        order_label = "Triplets"
    elif m == 4:
        order_int = m
        rder_label = "Quadruplets"
    elif m == 5:
        order_int = m
        order_label = "Quintuplets"
    else:
        print("\n** Error: Integer value between 1",
              "5 not supplied for m.**\n")
        return
        
    #    
    # Check and set save location.
    if savelocation == None:
        savelocation = \
           ("Mutual information saves/"+\
            order_label)
        print("\nNo save location provided." +
              "\n...Defaulting to ~/" + savelocation)    
    #
    # Check encoding. If not provided, 
    # default o OR encoding.
    
    # match encoding:
    #     case None:
    #         encoding_label = "ORencoding"
    #         print("\nNo encoding provided." +
    #               "\nDefaulting to '" + encoding + "' encoding.")
    #     case "or":
    #         encoding_label = "ORencoding"
    #     case "and":
    #         encoding_label = "ANDencoding"
    #     case "multi":
    #         encoding_label = "MULTIencoding"
    #     case _:
    #         print("\n** Error: Encoding value from ",
    #               "{'or', 'and', 'multi'} not provided.**\n")
    #         return
        
    if encoding == None:
        encoding_label = "ORencoding"
        print("\nNo encoding provided." +
              "\n...Defaulting to '" + encoding_label + "' encoding.")
    elif encoding == "or":
        encoding_label = "ORencoding"
    elif encoding == "and":
        encoding_label = "ANDencoding"
    elif encoding == "multi":
        encoding_label = "MULTIencoding"
    else:
        print("\n** Error: Encoding value from ",
              "{'or', 'and', 'multi'} not provided.**\n")
        return
        
    
    print("\n\n\n****************************************")  
    print(    "Calculating mutual information values...")
    # Define the m-way tuples of features sets as a numpy array. We will loop
    # through the rows of this array to create the feature sets.
    combins = \
        numpy.asarray(
            list(
                itertools.combinations(
                    featureSet_array.columns[featureSet_array.columns != 'person_id'],
                    order_int)
                )
            )
    # Instantiate specific storage for mutual information.
    featureSet_MI = \
        pandas.DataFrame(columns = ['Feature set', 'Mutual information'])
    # Instantiate batch number.
    batch = 0
    # Instantiate tally of feature sets that are dropped due to low entropy.
    drop_tally = 0
    
    # ## loop through the feature sets.
    for i_fs in range(len(combins)):
                
        # Define the feature set value.
        var_vals = featureSet_array[combins[i_fs]]
        binary_var = var_vals.all(True)
        
        # Calculate the mutual information for the feature set.
        f_MI = sklearn.metrics.mutual_info_score(binary_var, caseness_array['CMHD'])

        if f_MI < entropy_caseness:
            drop_tally += 1
            continue
        else:
            # Name the feature set.
            # ...
            # Store the name and mutual information value.
            featureSet_MI.loc[len(featureSet_MI)] = name_var, f_MI

        if len(featureSet_MI) > 9:
                # Increment batch.
                batch += 1

                # Make an interim save of results.
                featureSet_MI.to_csv(savelocation +
                                  order_label + "_" +
                                  encoding_label + "_" +
                                  "_batch" + \
                                  str(batch) + \
                                  ".csv", index = False)
                # Instantiate new storage.
                featureSet_MI = \
                    pandas.DataFrame(columns = ['Feature set', 'Mutual information'])


    # Increment counter.
    batch += 1

    # Final save.
    if len(featureSet_MI) != 0:
        featureSet_MI.to_csv(savelocation +
                          order_label + "_" +
                          encoding_label + "_" +
                          "_batch" + \
                          str(batch) + \
                          ".csv", index = False)

    # Feedback messages.
    print("...\n")
    print(str(batch), "batch(es) of feature sets processed.")
    print(str(drop_tally), "/",
          str(len(combins)),
          "feature sets dropped due to low entropy.")
    print("****************************************")  

### Mutual information of individual feature sets and the caseness variable.

In [244]:
featuresetmi(featureArray = featureSet_array,
            m = 1,
            encoding = "or")


No save location provided.
...Defaulting to ~/Mutual information saves/Individuals



****************************************
Calculating mutual information values...
...

1 batch(es) of feature sets processed.
216 / 216 feature sets dropped due to low entropy.
****************************************


### Mutual information of pair-composite feature sets and the caseness variable.

In [246]:
featuresetmi(featureArray = featureSet_array,
            m = 2,
            encoding = "or")


No save location provided.
...Defaulting to ~/Mutual information saves/Pairs



****************************************
Calculating mutual information values...
...

1 batch(es) of feature sets processed.
23220 / 216 feature sets dropped due to low entropy.
****************************************


### Mutual information of triplet-composite feature sets and the caseness variable.

In [240]:
featuresetmi(featureArray = featureSet_array,
            m = 3,
            encoding = "or")


No value for m provided.
...Default value of m = 1 will be used.

No save location provided.
...Defaulting to /Mutual information saves/Individuals

No encoding provided.
...Defaulting to 'ORencoding' encoding.



****************************************
Calculating mutual information values...
...

1 batch(es) of feature sets processed.
60 / 216 feature sets dropped due to low entropy.
****************************************


### Mutual information of quadruplet-composite feature sets and the caseness variable.

In [240]:
featuresetmi(featureArray = featureSet_array,
            m = 4,
            encoding = "or")


No value for m provided.
...Default value of m = 1 will be used.

No save location provided.
...Defaulting to /Mutual information saves/Individuals

No encoding provided.
...Defaulting to 'ORencoding' encoding.



****************************************
Calculating mutual information values...
...

1 batch(es) of feature sets processed.
60 / 216 feature sets dropped due to low entropy.
****************************************


### Mutual information of quintuplet-composite feature sets and the caseness variable.

In [240]:
featuresetmi(featureArray = featureSet_array,
            m = 5,
            encoding = "or")


No value for m provided.
...Default value of m = 1 will be used.

No save location provided.
...Defaulting to /Mutual information saves/Individuals

No encoding provided.
...Defaulting to 'ORencoding' encoding.



****************************************
Calculating mutual information values...
...

1 batch(es) of feature sets processed.
60 / 216 feature sets dropped due to low entropy.
****************************************


# Initial results

__\*__ _Note: these initial results were calculated using a previous version of the script where the mutual information of all feature sets was saved. This approach was dropped in favour of only saving mutual information values for feature sets whose mutual information with the caseness variable is greater than the entropy of the caseness variable._
<br/><br/><br/>

All individual feature sets score very low for two-way mutual information: all less than 0.05. A $I_{2}=0.05$ represents 7.9% of the theoretical maximal situatons where the feature set either _exactly is_ the caseness variable or _is exactly not_ the variable. The top five individal feature sets (which have $I_{2}≥0.033$) are defined as having at least one recording of the following SNOMED CT codes in their primary-care electronic health records:

| SNOMED code | Feature set | Topic | Mutual Information | Scaled mutual information | Odds ratio | P(CMHD given X=1) | P(CMHD given X=0) |
| ----------- | ----------- | ----- | ------------------ | ------------------------- | ---------- | ----------------- | ----------------- |
| 314530002 | Medication review done | Medication | ~0.055 | 7.9% | 8.5 | 4.1 | 26.8 |
| 182888003 | Medication requested  | Medication | ~0.035 | 5.0% | 4.9 | 7.9 | 29.5 |
| 1018251000000107 | Serum alanine aminotransferase level (observable entity) | Liver test | ~0.033 | 4.7% | 5.5 | 4.7 | 21.6 |
| 1000621000000104 | Serum alkaline phosphatase level | Liver test | ~0.033 | 4.7% | 5.5 | 4.7 | 21.5 |
| 1022791000000101 | TSH (thyroid stimulating hormone) level | Endocrine | ~0.033 | 4.7% | 5.0 | 5.4 | 22.5 |

The paradoxes of commonly-reported classification statistics are clearly shown. A medication _review_ has the largest odds ratio but the probability of having a record of the caseness variable given that medication was _requested_ is higher. One might propose that a record of a medication request is a better indicator than a record of a medication review, if they prefer the probability statistic over the odds-ratio statistic. But when we look at the probability of the caseness variable given that there is _no_ record of medication being requested, we see that this is also the largest of the top five feature sets! The odds ratio tries to balance these ambiguous probability statistics but it is, therefore, harder to interpret. Note, the odds ratio for a record of a medication _review_ scores better than for a record of medication _request_ because the distinction between the ambiguous probabilities is greater (multiplicatively).

The scaled mutual information is simply a percentage measure of how much the caseness variable is described by the feature set (in terms of information). Unlike the odds ratio, it will give the same value whether the odds are multiplicatively greater or less than equal - e.g. $I_{2}$ will be the same for $OR = 4.0$ and $OR = 0.25$  - so it only measures magnitude of association.